# 목표

- csv 파일을 읽어서 torchtext를 사용하여 데이터를 신경망에 입력가능한 꼴로 바꾸기
(Field, Iterator, train,test, evaluation and prediction)
- base line으로 Naive Bayes classification 구현
- 한국어 데이터 전처리를 위한 함수를 만들고 이를 torchtext에 통합하기 
- 제시된 여러 모델을 사용하여(transformers 제외) 성능을 향상 시키기
- training, evaluation 한 것을 test 데이터에 적용하여 성능을 보이기.
- predict를 사용하여 제시된 기사들의 분류 결과를 보이기

- 참고 사이트
    - https://pytorch.org/text/
    - http://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/
    - https://github.com/pytorch/text

## 내용

- 첨부된 BalancedNewsCorpus_train.csv, BalancedNewsCorpus_test.csv는 국어원 뉴스 자료에서 9개 분야의 신문별 균형을 맞춘 자료로, 학습용 9,000개 시험용 1800 자료가 있는 파일이다.
- 이 파일을 가지고 https://github.com/bentrevett/pytorch-sentiment-analysis 에 있는 pytorch sentiment analysis의 방법을 따라 한국어 뉴스기사 분류기를 만들어라
- 한국어 선처리를 위해 함수를 만들어 이를 torchText에 통합하여 사용. preprocessing은 다양한 방법으로 가능함.
- baseline으로 Naive Bayes를 사용하고 Neural Network를 사용하는 모델이 얼마나 더 성능의 향상을 이루는지 보여라..
- https://github.com/bentrevett/pytorch-sentiment-analysis 에 제시된 방법 중 가장 성능이 좋은 방법을 사용할 수 있음. **단 이 과제에서는 외부 임베딩과, transformers를 사용하는 방법은 적용하지 말것**
- Evaluation, Test 성능을 정리하고, 이렇게 학습한 모델로 제시된 User Input에서 제시된 문장의 출력과 정답을 비교 분석하라.
- 화일 이름은 MidTermProject_DS(or CL)_Group X
- 조원 이름 명시

## General
- 마감: 10월 21일 목요일 오후 12시!
- 이 노트북 화일에 이름을 변경하여 작업하고 제출. 제출시 화일명을 Assignment3_[DS또는 CL]_학과_이름.ipynb
- 화일에 각 조원 이름 명시
- 코드, 또는 셀 마다 자세한 설명 요함

## 정리
- 구현한 시스템의 성능을 정리 (프리프로세싱 방법, 사용한 모델, 테스트 성능 등)


In [21]:
## Your code starts
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext.legacy import data, datasets

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import json

import time

import re
from eunjeon import Mecab

csv 파일을 읽어서 torchtext를 사용하여 데이터를 신경망에 입력가능한 꼴로 바꾸기 (Field, Iterator, train, test, evaluation and prediction)

base line으로 Naive Bayes classification 구현

한국어 데이터 전처리를 위한 함수를 만들고 이를 torchtext에 통합하기

제시된 여러 모델을 사용하여(transformers 제외) 성능을 향상 시키기

training, evaluation 한 것을 test 데이터에 적용하여 성능을 보이기.

predict를 사용하여 제시된 기사들의 분류 결과를 보이기

In [2]:
## preprocessing
# torchtext는 target으로 int 만을 받는다. 
# 따라서 label column 을 만들고, Topic 에 대응하는 index number를 저장해줬다. 
train = pd.read_csv('BalancedNewsCorpus_train.csv')
test  = pd.read_csv('BalancedNewsCorpus_test.csv')

Topic_list = list(set(train['Topic']))
train['Label'] = [Topic_list.index(Topic) for Topic in train['Topic']]
test['Label']  = [Topic_list.index(Topic) for Topic in test['Topic']]

train.to_csv('BalancedNewsCorpus_train_labeled.csv', index = False)
test.to_csv('BalancedNewsCorpus_test_labeled.csv', index = False)

train_labeled = pd.read_csv('BalancedNewsCorpus_train_labeled.csv')
test_labeled = pd.read_csv('BalancedNewsCorpus_test_labeled.csv')
train_labeled.head()

,filename,date,NewsPaper,Topic,News,Label
0,NLRW1900000141,20170324,부산일보,스포츠,"<p> 야구 종가, 마침내 정상에 서다 </p> <p> '야구 종가' 미국이 푸에르...",8
1,NPRW1900000003,20110209,한국경제신문사,정치,"<p> 외통위 27명중 15명 ""FTA 추가협상안만 처리"" </p> <p> 국회 외...",4
2,NLRW1900000144,20100406,영남일보,사회,"<p> 한나라 ""地選후보, 희망연대 당원 구함"" 공천변수 작용 주목 </p> <p>...",3
3,NLRW1900000064,20100804,광주매일신문,스포츠,<p> 모처럼 살아난 ‘CK포’ 7타점 합작 </p> <p> KIA 12 3 LG ...,8
4,NLRW1900000070,20160615,광주매일신문,문화,<p> 亞문화전당서 동방의 등불 만나다 </p> <p> “일찍이 아시아의 황금 시기...,2


In [15]:
## define torchtext.data.Field
# tokenizer로는 은전한닢 프로젝트의 Mecab 을 사용했다.
tokenizer = Mecab()

# define stopwords
news_list = list(train['News']) + list(test['News'])
stopword_set = set()
for news in news_list:
    stopword_set = stopword_set.union(set(re.findall(r'[\W]', news))) # 특수기호
stopword_list = list(stopword_set)

with open("stopwords.json", 'r') as f: # frequently used stopwords (https://www.ranks.nl/stopwords/korean)
    stopwords = json.load(f)
    stopword_list += stopwords
stopword_list += ['p', '</', 'ㆍ','0','1','2','3','4','5','6','7','8','9'] # BOS/EOS and single digit number

print('----- stopword_list ----- \n', stopword_list[:50], '...etc')

NEWS = data.Field(
    sequential = True,
    use_vocab = True,
    is_target = False,
    tokenize = tokenizer.morphs,
    batch_first = False, #!#
    stop_words = stopword_list
)

LABEL = data.Field(
    sequential = False,
    use_vocab = False,
    is_target = True,
    dtype = torch.float
)

fields = {'News' : ('news', NEWS), 'Label' : ('label', LABEL)}

----- stopword_list ----- 
 ['·', '㎎', '㎒', '*', '☞', '■', '＂', '“', '♣', '㎜', '\x8b', '，', '~', '㈜', '@', '☎', '㎝', '●', '㎚', '㏈', '［', '┃', '↓', '/', '”', '㎃', '㎥', ':', '\x9f', '～', ';', '√', '＿', '㎹', '\uf0a0', '.', '—', '》', '’', '㎞', '<', '★', '＠', '|', '➢', '™', '㏃', '▋', '♡', '‥'] ...etc


In [37]:
train_data, test_data = data.TabularDataset.splits(
    path  = './',
    train = 'BalancedNewsCorpus_train_labeled.csv',
    test  = 'BalancedNewsCorpus_test_labeled.csv',
    format = 'csv',
    fields = fields
)

NEWS.build_vocab(train_data, max_size = 35000, min_freq = 10)
print(NEWS.vocab.itos)

['<unk>', '<pad>', '다', '는', '은', '고', '있', '한', '했', '도', '적', '해', '인', '만', '수', '기', '지', '게', '할', '며', '되', '원', '한다', '었', '말', '면', '않', '된', '명', '면서', '던', '받', '대', '없', '지역', '전', '중', '됐', '성', '았', '개', '다는', '화', '억', '된다', '시', '한국', '사업', '주', '대한', '라고', '10', '지난', '위해', '시장', '통해', '겠', '대표', '기업', '보', '위', '내', '라는', '때문', '보다', '다고', '같', '지원', '씨', '권', '정부', '될', '안', '많', '밝혔', '의원', '더', '세계', '개발', '당', '경우', '후보', '점', '지난해', '간', '가능', '계획', '두', '경기', '라', '사람', '최근', '차', '도록', '20', '위한', '미국', '경제', '돼', '국내', '문화', '올해', '관련', '좋', '높', '기술', '번', '특히', '서울', '산업', '천', '대해', '해야', '문제', '대통령', '진행', '관계자', '광주', '이날', '데', '업체', '부산', '현재', '가장', '30', '병원', '시작', '예정', '이후', '분', '팀', '다양', '중국', '도시', '라며', '운영', '필요', '제품', '대회', '이나', '11', '사회', '곳', '12', '뒤', '후', '회', '만들', '사용', '연구', '센터', '큰', '조', '결과', '조사', '추진', '는데', '활동', '상황', '참여', '새', '기록', '형', '선수', '신', '서', '세', '국민', '달', '방송', '김', '서비스', '건', '의료', '따르', '대상', '투자', '첫', '

In [32]:
batch_size = 64
train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, test_data),
    batch_size = batch_size,
    sort_key = lambda x: len(x.news), # https://github.com/pytorch/text/issues/474
    sort_within_batch = False
#     sort=False # https://github.com/pytorch/text/issues/474
)

In [33]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, news):
        embedded = self.embedding(news)
        output, hidden = self.rnn(embedded)
        return self.fc(hidden.squeeze(0))

    
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.news).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.news).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [34]:
# hyper parameters
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
N_EPOCHS = 5
LR = 1e-3

model = RNN(input_dim = len(NEWS.vocab), 
            embedding_dim = EMBEDDING_DIM, 
            hidden_dim    = HIDDEN_DIM, 
            output_dim    = 1)

optimizer = optim.SGD(model.parameters(), lr = LR)
criterion = nn.BCEWithLogitsLoss()

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 2m 48s
	Train Loss: -115.085 | Train Acc: 11.51%
	 Val. Loss: -172.036 |  Val. Acc: 11.05%
Epoch: 02 | Epoch Time: 1m 33s
	Train Loss: -431.207 | Train Acc: 11.86%
	 Val. Loss: -304.775 |  Val. Acc: 10.51%
Epoch: 03 | Epoch Time: 1m 21s
	Train Loss: -758.065 | Train Acc: 11.88%
	 Val. Loss: -419.518 |  Val. Acc: 10.51%
Epoch: 04 | Epoch Time: 1m 22s
	Train Loss: -1083.958 | Train Acc: 11.84%
	 Val. Loss: -609.666 |  Val. Acc: 10.51%
Epoch: 05 | Epoch Time: 1m 17s
	Train Loss: -1409.764 | Train Acc: 11.76%
	 Val. Loss: -861.279 |  Val. Acc: 10.99%


In [36]:
Topic_list

['미용/건강', 'IT/과학', '문화', '사회', '정치', '생활', '경제', '연예', '스포츠']

In [165]:
## Naive Bayes

In [ ]:
## Your choice of Neural Network Model

## User Input

####  뉴스 labels
    -  IT/과학': 0, '경제': 1, '문화': 2, '미용/건강': 3, '사회': 4, '생활': 5, '스포츠': 6, '연예': 7, '정치': 8

In [ ]:
def predict_news(model, sentence, min_len=5):

In [ ]:
## 아래 문장의 정답은 8-4-1-3-5-6-0-2-7
## 연예/문화, 정치/경제/사회/생활 등 명확히 구별되기 어려운 범주들이 있음...

sentence = "여러 차례 선거를 치르며 조직적인 지지모임과 온라인 팬덤을 보유한 이 지사에 비해 부족하다는 것이다. 윤석열 캠프에선 오차범위를 다투는 여론조사 지지율과는 별개로, ‘조직’에 있어선 아직도 채워야 할 부분이 많다는 이야기가 나온다. 윤석열 캠프 관계자는 “사람만 많이 모은다고 좋을 게 없다는 지적을 듣기도 하지만, 인구 1300만명의 지자체장인 이재명 지사에 비하면 많은 게 아닌 상황”이라고 말했다."
sentence = "여자친구가 이별을 통보하고 새 남자친구를 사귀자 지속적으로 찾아가 협박과 폭행을 가한 30대 남성이 실형을 선고 받았다. 창원지법 형사4단독 안좌진 부장판사는 상해, 주거침입, 폭행 등 혐의로 재판에 넘겨진 A(39)씨에게 징역 1년3개월을 선고했다고 10일 밝혔다.A씨는 지난해 4월부터 올해 2월까지 10개월 가량 사귄 B씨(30)가 이별을 통보하자 지난 3월 6일 B씨 집을 찾아가 욕설을 퍼붓고 B씨를 폭행한 혐의를 받고 있다."
sentence = "동탄신도시의 성공은 명실상부한 한국 1위의 기업 삼성전자를 빼놓고는 설명할 수 없다. 삼성전자는 수출의 20%를 담당하는 한국 경제의 심장이다. 삼성전자의 연구소와 공장은 세계 최고 수준의 연구인력과 협력업체를 끌어당기는 블랙홀이다.동탄신도시 인근에 삼성전자 기흥캠퍼스가 있고 화성캠퍼스가 신도시에 자리 잡고 있다. 삼성 화성캠퍼스에서는 메모리와 파운드리 반도체의 설계 및 생산이 이뤄지고 있다."
sentence = "샤워나 목욕 중에는 물, 샤워타올, 수건 등 균이 닿을 여지가 많다. 샤워를 하는 화장실에는 보통 변기도 함께 있어 배변 활동으로 나온 균이 공기 중을 돌아다니고 있다. 습기가 높아 곰팡이가 생기기도 좋은 환경이다. 화장실에 걸린 샤워타올과 수건이 제대로 건조되지 않은 채 화장실에 내내 있었다면 균이 있을 가능성이 크다. 이 균이 예방 접종 하면서 생긴 손상 부위에 닿으면 드물지만 침입해 감염증을 유발할 수 있다."
sentence = "식전주의 시간이다. 밥을 먹기 전에 마시는 술. 안주와 함께 먹지 않는 술. 술만으로 온전한 술. 이게 식전주다. 3시와 5시 사이는 식전주의 시간이기도 한 것이다. 이 시간에 마시는 식전주를 나는 꽤나 좋아한다. 술은 다 각각의 매력이 있고, 슬플 때도 기쁠 때도 지루할 때도 피곤할 때도 좋지만, 식전주의 시간에 마시는 식전주도 좋다. 주로 맥주이지만 가끔은 아페리티프(Aperitif·식전주)를 마신다."
sentence = "시리아전을 마친 뒤 9일 이란으로 출국한 한국 대표팀은 한국 시간 기준 10일 오전 1시경 테헤란 공항에 도착해 숙소인 파르시안 아자디 호텔로 이동했다. 이후 코로나19 PCR 검사를 진행했고, 결과가 나올 때까지 각자 방에서 격리한 채 대기할 예정이다. 한국은 역대 이란 원정에서 한차례도 승리하지 못한 채 2무 5패를 기록 중이다.  선수들이 좋은 컨디션을 유지할 수 있도록 전세기를 마련해 이란으로 향했다."
sentence = "애플의 아이폰13 시리즈가 지난 8일 국내 판매를 시작했다. 애플이 지난달 14일(현지시각) 신제품을 공개한 후 3주 만이다. 애플은 아이폰13의 두뇌에 해당하는 프로세서와 카메라 성능을 크게 개선했다고 밝혔다. 팀 쿡 애플 최고경영자(CEO)는 “역사상 최고의 아이폰이다”라고 했다. 하지만 전작인 아이폰12와 비교해 큰 차이를 느낄 수 없다는 부정적인 평가도 많다."
sentence = "극단 마실은 문화체육관광부와 지역문화진흥원 지원으로 '심청전-할머니의 비밀레시피' 온라인 만남 행사를 진행했다고 10일 밝혔다. 행사는 할머니만의 레시피로 함께 음식을 만들며 할머니의 이야기를 공유하고, 할머니를 주인공으로 한 짤막한 연극을 펼치는 순서로 진행됐다. 극단은 지역 내 관음사 연기 설화가 심청전과 연관 있는 점을 토대로 심청의 일생과 닮은 곡성 할머니들의 이야기를 2018년도부터 수집해 연극을 만들었다."
sentence = "‘놀면 뭐하니?+’에서는 유재석, 정준하, 하하, 신봉선, 미주의 깜짝 ‘꼬치꼬치 기자간담회’와 MBC 보도국 열혈 신입기자로 변신한 ‘뉴스데스크’ 특집이 시작됐다. ‘꼬치꼬치 기자간담회’에서는 정준하가 ‘스포츠 꼬치꼬치’ 기자로 변신, 시청자의 궁금증을 풀어주는 마성의 돌직구 질문을 던졌고, ‘놀면 뭐하니?+’ 멤버들은 솔직한 마음이 담긴 답변으로 큰 웃음과 훈훈함을 동시에 선사했다."
predict_news(model, sentence)